Test the LCOE/WACC

In [ ]:
import pypsa
import pandas as pd

In [ ]:
prefix = "/mnt/c/Users/scl38887/Documents/git/aldehyde"

Import network

In [ ]:
# Import from aldehyde
# run = "store_complete_6h_ws"#"highwacc_6h_ws"
# n = pypsa.Network(prefix + "/resources/pypsaearthsec/" + run + "/postnetworks/elec_s_10_ec_lc1.0_Co2L1.0_6H_2030_0.071_DF_0export.nc")


In [ ]:
# Import from pypsa-earth-sec
run = "highwacc_hem_1marg_1cap_nosmr_2000h_ws"
n = pypsa.Network(prefix + "/workflow/subworkflows/pypsa-earth-sec/results/" + run + "/postnetworks/elec_s_10_ec_lc1.0_Co2L0.10_2000H_2030_0.35_DF_0export.nc")
# n = pypsa.Network(prefix + "/workflow/subworkflows/pypsa-earth-sec/results/store_complete_6h_ws/postnetworks/elec_s_10_ec_lc1.0_Co2L1.0_6H_2030_0.071_DF_0export.nc")

Check characteristics of fossils

In [ ]:
n.generators.carrier.unique()

In [ ]:
n.generators.loc[n.generators.carrier == "coal"].p_nom_opt #.iloc[20, :]#, "marginal_cost"]

In [ ]:
n.generators.loc[n.generators.carrier == "coal", "marginal_cost"]

In [ ]:
n.generators.loc[n.generators.carrier == "oil", "marginal_cost"]

In [ ]:
#n.generators.loc[n.generators.carrier == "gas"] #, "marginal_cost"] 

In [ ]:
n.generators.carrier.unique()

Check objective

In [ ]:
n.objective/1e9 # billion euros

Check capital cost before `prepare_sector_network`

In [ ]:

n.generators.capital_cost[:20] #highwacc: 96085 €/MW onwind and 35602 €/MW solar

Check CO2

In [ ]:
(n.stores_t.p["co2 atmosphere"]*n.snapshot_weightings.generators[0]).sum()/1e6

Check n.statistics()

In [ ]:
n.statistics().loc["Generator"].round(2)

In [ ]:
# Get first index levels of multiindex
n.statistics().index.get_level_values(0).unique()

In [ ]:
n.statistics().loc["Link"].round(2)

Check "gas"

In [ ]:
n.generators[n.generators.carrier=="gas"].iloc[0]

Check network contraint

In [ ]:
n.global_constraints

Calculate LCOE

In [ ]:
generator_name = 'MA.10.3_1_AC coal'#'MA.1.1_1_AC onwind'  # Replace with the name or index of your generator

capital_cost = n.generators.loc[generator_name, 'capital_cost']
marginal_cost = n.generators.loc[generator_name, 'marginal_cost']
lifetime = 20

In [ ]:
lcoe = (capital_cost / lifetime + marginal_cost) / n.generators_t.p[generator_name].sum()
lcoe # Unit in €/MWh

In [ ]:
n.generators_t.p[generator_name].sum()

In [ ]:
n.global_constraints

In [ ]:
def calculate_annuity(n, r):
    """
    Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20, 0.05) * 20 = 1.6
    """
    if isinstance(r, pd.Series):
        return pd.Series(1 / n, index=r.index).where(
            r == 0, r / (1.0 - 1.0 / (1.0 + r) ** n)
        )
    elif r > 0:
        return r / (1.0 - 1.0 / (1.0 + r) ** n)
    else:
        return 1 / n

In [ ]:
calculate_annuity(20, 0.05)

In [ ]:
costs["capital_cost"] = (
    (
        calculate_annuity(costs["lifetime"], costs["discount rate"])
        + costs["FOM"] / 100.0
    )
    * costs["investment"]
    * Nyears
)

Plot network

In [ ]:
import pypsa, os
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd

import warnings
from shapely.errors import ShapelyDeprecationWarning

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
plt.rc("figure", figsize=(10, 8))